<a href="https://colab.research.google.com/github/sujay-dsa/ComputerVision/blob/master/MNIST/EVA5_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import pytorch modules

from __future__ import print_function 
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module): # class Net is extending the nn.Module class from pytorch
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -28x28x1 OUtput-28x28x32  RF - 3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input -28x28x32 OUtput-28x28x64  RF - 5
        self.pool1 = nn.MaxPool2d(2, 2) #input -28x28x64 OUtput-14x14x64  RF -6
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # input - 14x14x64, output - 14x14x128, rf-8
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # input - 14x14x128, output - 14x14x256, rf-10
        self.pool2 = nn.MaxPool2d(2, 2) # input=14x14x256 output - 7x7x256 rf -11
        self.conv5 = nn.Conv2d(256, 512, 3) # input 7x7x256, output - 5x5x512 - rf 13
        self.conv6 = nn.Conv2d(512, 1024, 3) # input 5x5x512, output - 3x3x1024 - rf 15
        self.conv7 = nn.Conv2d(1024, 10, 3) # input 3x3x1024, output - 1x1x10 - rf 17

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10) # this method is similar to numpy's reshape. It gives us 1 row, 10 columns.
        return F.log_softmax(x)

In [29]:
# This cell is for experiments
class Net(nn.Module): # class Net is extending the nn.Module class from pytorch
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=0) #input -28x28x1 OUtput-28x28x32  RF - 3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=0) #input -28x28x32 OUtput-28x28x64  RF - 5
        self.pool1 = nn.MaxPool2d(2, 2) #input -28x28x64 OUtput-14x14x64  RF -6
        self.conv3 = nn.Conv2d(64, 128, 3, padding=0) # input - 14x14x64, output - 14x14x128, rf-8
        self.pool2 = nn.MaxPool2d(2, 2) # input=14x14x256 output - 7x7x256 rf -11
        self.conv4 = nn.Conv2d(128, 256, 3, padding=0) # input - 14x14x128, output - 14x14x256, rf-10
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1) # input - 14x14x128, output - 14x14x256, rf-10
        
        
        self.conv6 = nn.Conv2d(512, 10, 3) # input 7x7x256, output - 5x5x512 - rf 13
        #self.conv6 = nn.Conv2d(512, 1024, 3) # input 5x5x512, output - 3x3x1024 - rf 15
        #self.conv7 = nn.Conv2d(512, 10, 3) # input 3x3x1024, output - 1x1x10 - rf 17

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv3(x)))
        x = F.relu(self.conv5(F.relu(self.conv4(x))))
        x = F.relu(self.conv6(x))
        #x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available() # check if nvidia gpu is available. if not use cpu
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:


torch.manual_seed(1) # seed for random number generation for our kernels
batch_size = 128 # this indicates with how many images at a time training happens

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {} # for CUDA enabled GPU pin_memory=True enables faster data transfer. 
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)) # normalizes input data. Here the first param is the mean and second is the std deviation. We pass one value because this is single channel input
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs) #shuffle=true ensures data is shuffled every epoch. Helps with getting a more generic solution


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
# Using a Stochastic Gradient Descent algorithm to modify values. lr refers to learning rate, momentum indicates how aggressively 
# we want to descent to happen. The risk here would be overshooting the global minima if it is too high. 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) 

for epoch in range(1, 2): # training for 2 epochs.
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.7311, Accuracy: 3805/10000 (38%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.7877, Accuracy: 7662/10000 (77%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.5290, Accuracy: 7778/10000 (78%)



loss=0.4086296260356903 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.18it/s]



Test set: Average loss: 0.5266, Accuracy: 7774/10000 (78%)

